## Introduction to the problem 

In a certain organization, a recruitment is being conducted to admit new members. For every two members, a mentor from that organization must be assigned. Each mentor interviews the newly admitted member in advance on a speed dating basis. In this way, recruits with mentors rate each other on a scale of 0 to 3. It is necessary to assign such baby members to mentors that the sum of assigned scores is as high as possible. 


# Solution 

Imagine a weighted graph with edges of a given capacity, consisting of 3n+2 vertices, where n is the number of mentors. The first of these will represent the source and will be accessed by 2n units. From the source there will be n edges to n vertices, each with capacity 2 and cost 0. These vertices represent the given mentors. From each of vertices 1 - n there will be an edge to vertices n+1-3n, which represent baby members. Each such edge will have a throughput equal to 1 and a cost equal to the sum of the ratings that the members have given between each other. The last vertex will act as an outlet, to which edges with a throughput of 1 and a cost of 0 will lead from vertices denoting baby memebers.  

With such a graph we can perform an algorithm calculating min cost flow, whose implementation can be found in Python. In this way we will find the most optimal solution. 

# Example 


![](picture2.png)
![](picture1.png)


# Libraries 


In [12]:

from __future__ import print_function
from ortools.graph import pywrapgraph

# Constants 

In [13]:
MAX_MARK = 3
NUMBER_ANGELS = 8
NUMBER_BABIES = 16
CAPACITY_1 = 1
CAPACITY_2 = 2
COST_0 = 0
COST_LOAD = 20
ID_0 = 0
SUPPLIE_0 = 0
MULTIPLIER_MENTORS_MARK = 2
MULTIPLIER_BABIES_MARK = 3

# Create Graph

#### Edge ends 

In [14]:
def create_start_nodes(number_angels, number_baybes):
    nodes = []
    for i in range (number_angels):
        nodes.append(ID_0)
    for i in range (1,number_angels+1):
        for j in range (0, number_baybes):
            nodes.append(i)
    for i in range (number_angels+1,number_angels*3+1):
            nodes.append(i)
    return nodes

def create_end_nodes(number_angels, number_baybes):
    nodes = []
    for i in range(1,number_angels+1 ):
        nodes.append(i)
    for i in range (0,number_angels):
        for j in range (number_angels+1, number_angels+number_baybes+1):
            nodes.append(j)
    for i in range (number_baybes):
        nodes.append(number_angels+number_baybes+1)
    return nodes


#### Capacities 

In [15]:
def create_capacities(number_angels, number_baybes ):
    nodes = []
    for i in range (number_angels):
        nodes.append(CAPACITY_2)
    for i in range (number_angels*number_baybes+number_baybes):
        nodes.append(CAPACITY_1)
    return nodes


#### Supplies 

In [16]:
def create_supplies(number_angels,number_baybes):
    nodes = []
    nodes.append(number_baybes)
    for i in range (number_angels+number_baybes):
        nodes.append(SUPPLIE_0)
    nodes.append(-number_baybes)
    return nodes

#### Costs

In [17]:
def create_costs( number_angels,number_baybes, angels_marks,beybes_marks):

    nodes = []
    for i in range (number_angels):
        nodes.append(COST_0)
    for i in range (number_angels*number_baybes):
        if angels_marks[i] == 1 or beybes_marks[i]==1:
            nodes.append(COST_LOAD)
        else:
            nodes.append(MULTIPLIER_MENTORS_MARK*(MAX_MARK-angels_marks[i])+MULTIPLIER_BABIES_MARK*(MAX_MARK-beybes_marks[i]))
    for i in range ( number_baybes):
        nodes.append(COST_0)
    return nodes

# INPUT 

In [20]:
number_mentors = 8
number_babies = 16

angels_marks = [2,3,2,4,3,2,1,3,3,5,3,5,2,5,3,5,
                1,3,2,4,3,1,5,3,1,1,3,5,2,2,3,2,
                5,4,5,4,3,5,4,5,5,5,3,5,5,5,3,5,
                3,3,4,2,3,2,3,4,2,3,3,4,4,2,3,4,
                3,1,4,4,3,1,4,3,1,4,3,5,5,1,3,5,
                5,5,5,5,3,5,5,5,5,5,3,5,5,5,3,5,
                4,4,5,5,3,1,3,2,2,2,3,4,5,2,3,2,
                5,5,3,5,3,2,3,3,1,3,3,3,4,1,3,3]

babies_marks = [5,3,5,3,4,5,5,5,
                5,3,3,4,5,4,4,5,
                4,5,4,4,4,3,5,4,
                4,3,4,3,3,5,5,4,
                3,3,3,3,3,3,3,3,
                2,3,4,4,5,1,4,5,
                4,3,5,4,4,5,4,2,
                2,4,2,5,5,4,4,5,
                5,4,5,3,4,5,4,3,
                5,3,4,4,5,3,3,4,
                3,3,3,3,3,3,3,3,
                3,2,4,2,5,5,4,4,
                5,3,4,3,5,5,5,4,
                5,3,3,5,4,3,3,4,
                5,3,3,3,3,3,3,3,
                3,3,4,3,4,4,3,4]

# MAIN 

In [19]:
start_nodes = create_start_nodes(number_mentors, number_babies)
end_nodes   = create_end_nodes(number_mentors, number_babies)
capacities  = create_capacities(number_mentors, number_babies)
supplies = create_supplies(number_mentors, number_babies)
unit_costs  = create_costs(number_mentors, number_babies, angels_marks, babies_marks)
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

for i in range(0, len(start_nodes)):
    min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i],capacities[i], unit_costs[i])

for i in range(0, len(supplies)):
    min_cost_flow.SetNodeSupply(i, supplies[i])

if min_cost_flow.Solve() == min_cost_flow.OPTIMAL:
    for i in range(min_cost_flow.NumArcs()):
        if i >= number_mentors and i< number_mentors*number_babies+number_mentors and min_cost_flow.Flow(i)==1:
            print("Mentor: ", min_cost_flow.Tail(i)," Baby: ", min_cost_flow.Head(i) - number_mentors)
else:
    print('There was an issue with the min cost flow input.')






Mentor:  1  Baby:  8
Mentor:  1  Baby:  9
Mentor:  2  Baby:  7
Mentor:  2  Baby:  15
Mentor:  3  Baby:  10
Mentor:  3  Baby:  16
Mentor:  4  Baby:  3
Mentor:  4  Baby:  6
Mentor:  5  Baby:  11
Mentor:  5  Baby:  13
Mentor:  6  Baby:  4
Mentor:  6  Baby:  14
Mentor:  7  Baby:  5
Mentor:  7  Baby:  12
Mentor:  8  Baby:  1
Mentor:  8  Baby:  2
